<a href="https://colab.research.google.com/github/pravsels/DistilLM/blob/main/HF_Training_Script.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install datasets transformers accelerate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 507.1/507.1 kB 5.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 270.9/270.9 kB 7.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 8.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 7.5 MB/s eta 0:00:00


In [ ]:
! apt install git-lfs

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
git-lfs is already the newest version (3.0.2-1ubuntu0.2).
0 upgraded, 0 newly installed, 0 to remove and 29 not upgraded.


In [ ]:
from huggingface_hub import notebook_login
notebook_login()

In [ ]:
import transformers
from datasets import load_dataset
import math
from transformers import AutoTokenizer, AutoModelForCausalLM, Trainer, TrainingArguments

In [ ]:
wikitext_dataset = load_dataset('wikitext',
                                'wikitext-2-raw-v1')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating test split:   0%|          | 0/4358 [00:00<?, ? examples/s]

Generating train split:   0%|          | 0/36718 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/3760 [00:00<?, ? examples/s]

In [ ]:
wikitext_dataset['train'][11]

{'text': " Troops are divided into five classes : Scouts , Shocktroopers , Engineers , Lancers and Armored Soldier . Troopers can switch classes by changing their assigned weapon . Changing class does not greatly affect the stats gained while in a previous class . With victory in battle , experience points are awarded to the squad , which are distributed into five different attributes shared by the entire squad , a feature differing from early games ' method of distributing to different unit types . \n"}

In [ ]:
model_name = 'distilgpt2'
# using the same tokenizer that was used for training the model
tokenizer = AutoTokenizer.from_pretrained(model_name, use_fase=True)

config.json:   0%|          | 0.00/762 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

In [ ]:
def tokenize_func(examples):
  return tokenizer(examples['text'])

In [ ]:
tokenized_dataset = wikitext_dataset.map(tokenize_func,
                                         batched=True,
                                         num_proc=8,
                                         remove_columns=['text'])

Map (num_proc=8):   0%|          | 0/4358 [00:00<?, ? examples/s]

Map (num_proc=8):   0%|          | 0/36718 [00:00<?, ? examples/s]

Map (num_proc=8):   0%|          | 0/3760 [00:00<?, ? examples/s]

In [ ]:
print(tokenized_dataset)
print(tokenized_dataset['train'][5])

DatasetDict({
    test: Dataset({
        features: ['input_ids', 'attention_mask'],
        num_rows: 4358
    })
    train: Dataset({
        features: ['input_ids', 'attention_mask'],
        num_rows: 36718
    })
    validation: Dataset({
        features: ['input_ids', 'attention_mask'],
        num_rows: 3760
    })
})
{'input_ids': [632, 1138, 351, 3967, 4200, 287, 2869, 837, 290, 373, 15342, 416, 1111, 4960, 290, 8830, 9188, 764, 2293, 2650, 837, 340, 2722, 41496, 2695, 837, 1863, 351, 281, 9902, 8313, 287, 3389, 286, 326, 614, 764, 632, 373, 635, 16573, 656, 15911, 290, 281, 2656, 2008, 11034, 2168, 764, 14444, 284, 1877, 4200, 286, 569, 18354, 7496, 17740, 2873, 837, 569, 18354, 7496, 17740, 6711, 373, 407, 36618, 837, 475, 257, 4336, 11059, 11670, 351, 262, 983, 705, 82, 9902, 8313, 373, 2716, 287, 1946, 764, 6343, 13, 44206, 561, 1441, 284, 262, 8663, 351, 262, 2478, 286, 569, 18354, 7496, 1058, 22134, 9303, 329, 262, 14047, 604, 764, 220, 198], 'attention_mask': [1, 1, 1,

In [ ]:
# setting context length of out inputs
# block_size = tokenizer.model_max_length
block_size = 128

In [ ]:
def group_text_func(examples):
  # for every key ('train', 'test' etc.), concatenate the texts
  concat_examples = {key: sum(examples[key], []) for key in examples.keys()}
  total_len = len(concat_examples[list(examples.keys())[0]])
  total_len = (total_len // block_size) * block_size

  # split by chunks of block_size
  result = {
      key: [text[i : i + block_size] for i in range(0, total_len, block_size)] for key, text in concat_examples.items()
  }
  result['labels'] = result['input_ids'].copy()

  return result

In [ ]:
lm_dataset = tokenized_dataset.map(group_text_func,
                                   batched=True,
                                   batch_size=512,
                                   num_proc=8)
lm_dataset

Map (num_proc=8):   0%|          | 0/4358 [00:00<?, ? examples/s]

Map (num_proc=8):   0%|          | 0/36718 [00:00<?, ? examples/s]

Map (num_proc=8):   0%|          | 0/3760 [00:00<?, ? examples/s]

DatasetDict({
    test: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 2205
    })
    train: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 18648
    })
    validation: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 1928
    })
})

In [ ]:
print('input ids : ', lm_dataset['train'][0]['input_ids'][:5])
print('attention mask : ', lm_dataset['train'][0]['attention_mask'][:5])
print('label ids : ', lm_dataset['train'][0]['labels'][:5])

input ids :  [796, 569, 18354, 7496, 17740]
attention mask :  [1, 1, 1, 1, 1]
label ids :  [796, 569, 18354, 7496, 17740]


In [ ]:
# load model
model = AutoModelForCausalLM.from_pretrained(model_name)

model.safetensors:   0%|          | 0.00/353M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

In [ ]:
training_args = TrainingArguments(
  f"{model_name}-finetuned-wikitext2",
  evaluation_strategy = "epoch",
  learning_rate=2e-5,
  # per_device_train_batch_size=8,
  # per_device_eval_batch_size=8,
  # gradient_accumulation_steps=2,
  weight_decay=0.01,
  push_to_hub=True
)

In [ ]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=lm_dataset['train'],
    eval_dataset=lm_dataset['validation']
)

In [ ]:
trainer.train()

Epoch,Training Loss,Validation Loss
1,3.756300,3.663765
2,3.628000,3.645566
3,3.589700,3.641406


TrainOutput(global_step=6993, training_loss=3.6731724749302876, metrics={'train_runtime': 398.2176, 'train_samples_per_second': 140.486, 'train_steps_per_second': 17.561, 'total_flos': 1827248176889856.0, 'train_loss': 3.6731724749302876, 'epoch': 3.0})

In [ ]:
eval_results = trainer.evaluate()
print(f"Perplexity score : {math.exp(eval_results['eval_loss']):.2f}")

Perplexity score : 38.15


In [ ]:
trainer.push_to_hub()

model.safetensors:   0%|          | 0.00/328M [00:00<?, ?B/s]

events.out.tfevents.1705754260.90d56b2707cd.5917.1:   0%|          | 0.00/7.80k [00:00<?, ?B/s]

Upload 3 LFS files:   0%|          | 0/3 [00:00<?, ?it/s]

events.out.tfevents.1705754663.90d56b2707cd.5917.2:   0%|          | 0.00/359 [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/pravsels/distilgpt2-finetuned-wikitext2/commit/eb88e0d0b1f27026a53fc0a19ec90d09206629a9', commit_message='End of training', commit_description='', oid='eb88e0d0b1f27026a53fc0a19ec90d09206629a9', pr_url=None, pr_revision=None, pr_num=None)

In [ ]:
# load_model_hub = AutoModelForCausalLM.from_pretrained("pravsels/distilgpt2-finetuned-wikitext2")

In [ ]:
from transformers import DataCollatorForLanguageModeling, AutoModelForMaskedLM

# masked language modelling
model_name = "distilroberta-base"

tokenizer = AutoTokenizer.from_pretrained(model_name,
                                          use_fast=True)

tokenized_dataset = wikitext_dataset.map(tokenize_func,
                                         batched=True,
                                         num_proc=8,
                                         remove_columns=["text"])



lm_dataset = tokenized_dataset.map(
    group_text_func,
    batched=True,
    batch_size=512,
    num_proc=8
)

model = AutoModelForMaskedLM.from_pretrained(model_name)

training_args = TrainingArguments(
    f"{model_name}-finetuned-wikitext2",
    evaluation_strategy = "epoch",
    learning_rate=2e-5,
    weight_decay=0.01,
    push_to_hub=True,
)

# we do this for random masking of tokens
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer,
                                                mlm_probability=0.15)


trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=lm_dataset["train"],
    eval_dataset=lm_dataset["validation"],
    data_collator=data_collator,
)

trainer.train()

config.json:   0%|          | 0.00/480 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

Map (num_proc=8):   0%|          | 0/4358 [00:00<?, ? examples/s]

...

Map (num_proc=8):   0%|          | 0/3760 [00:00<?, ? examples/s]

model.safetensors:   0%|          | 0.00/331M [00:00<?, ?B/s]

Some weights of the model checkpoint at distilroberta-base were not used when initializing RobertaForMaskedLM: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing RobertaForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
You're using a RobertaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss
1,2.083600,1.884397
2,1.985000,1.866782
3,1.952000,1.868227


TrainOutput(global_step=7209, training_loss=2.034787376341368, metrics={'train_runtime': 450.7423, 'train_samples_per_second': 127.922, 'train_steps_per_second': 15.994, 'total_flos': 1911743342023680.0, 'train_loss': 2.034787376341368, 'epoch': 3.0})

In [ ]:
eval_results = trainer.evaluate()
print(f"Perplexity score : {math.exp(eval_results['eval_loss']):.2f}")

Perplexity score : 6.29


In [ ]:
trainer.push_to_hub()

Upload 3 LFS files:   0%|          | 0/3 [00:00<?, ?it/s]

events.out.tfevents.1705754872.90d56b2707cd.5917.3:   0%|          | 0.00/7.65k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/329M [00:00<?, ?B/s]

events.out.tfevents.1705755329.90d56b2707cd.5917.4:   0%|          | 0.00/359 [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/pravsels/distilroberta-base-finetuned-wikitext2/commit/b347017e52f0e7ff283f77620183c01a7eb7a2be', commit_message='End of training', commit_description='', oid='b347017e52f0e7ff283f77620183c01a7eb7a2be', pr_url=None, pr_revision=None, pr_num=None)